In [6]:
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from datetime import date
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Definimos la ruta en donde queremos guardar los archivos
ruta = '/home/estadistico/Documents/Erick/Reportes diarios/Promesas Diarias'
#Definimos la ruta donde estan las asignaciones
ruta_asig_baz = '/home/estadistico/Documents/Erick/Banco Azteca/Asignacion csv'
mes2 = 'Diciembre2020'

#Agregamos las variables a ocupar
servidor = '192.168.15.12'
puerto = int('3306')
usuario = 'estadisticas'
contraseña = 'estadisticas8474'
base = 'procesos_externos'

#Hacemos la conexion con el servidor\n",
try:
    conn = mysql.connector.connect(user=usuario,
                               password=contraseña,
                               host=servidor,
                               port=3306,
                               database=base)
    conn.set_charset_collation('latin1')
except mysql.Error as e:
    print("Failed to execute stored procedure: {}".format(error))

cursor = conn.cursor()


# Liverpool

In [11]:
#Hacemos la consulta de la asignacion
sql_asig_liv = cursor.callproc('liverpool_rpt_asignacion_activa',)
for result in cursor.stored_results():
    asignacion_liv = pd.DataFrame(result.fetchall())
asignacion_liv = asignacion_liv.iloc[:,[0,1,3,24,6,26,8]]
asignacion_liv.columns = ['firma_id','unegocio_id','credito','estado','rfc','division','saldo_recuperar']
asignacion_liv['credito'] = pd.to_numeric(asignacion_liv['credito'])
asignacion_liv['saldo_recuperar'] = pd.to_numeric(asignacion_liv['saldo_recuperar'])
#asignacion_liv.head(3)

In [14]:
#Obtenemos datos de las promesas de Liverpool
pivot_liv = pd.pivot_table(asignacion_liv,index=['division'],values=['credito','saldo_recuperar'],aggfunc=['count',np.sum],margins=True,dropna=False)
pivot_liv = pd.DataFrame(pivot_liv.to_records())
pivot_liv = pivot_liv.iloc[:,[0,1,4]]
pivot_liv.columns = ['Division','NumeroPromesas','SaldoRecuperar']
pivot_liv.fillna(0,inplace=True)
#pivot_liv[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_liv[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_liv

,division,"('count', 'credito')","('sum', 'saldo_recuperar')"
0,,42,3.549011e+05
1,DEPN,71,4.884212e+05
2,GAA1,1,1.249341e+04
3,GAA3,1,3.389358e+04
4,JGS3,2150,3.668020e+07
5,NOQ1,1,0.000000e+00
6,RCC4,1211,1.928807e+07
7,REM2,37123,6.702653e+08
8,REM4,67858,1.349509e+09
9,REM9,811,3.822455e+06


# Credifiel

In [18]:
#Hacemos la consulta de la asignacion
sql_asig_cred = cursor.callproc('credifiel_rpt_asignacion_activa',)
for result in cursor.stored_results():
    asignacion_cred = pd.DataFrame(result.fetchall())
asignacion_cred = asignacion_cred.iloc[:,[1,3,32,8]]
asignacion_cred.columns = ['unegocio_id','credito','division','saldo_recuperar']
asignacion_cred['saldo_recuperar'] = pd.to_numeric(asignacion_cred['saldo_recuperar'])
asignacion_cred = asignacion_cred.loc[asignacion_cred['saldo_recuperar']>0]
#asignacion_brad['credito'] = pd.to_numeric(asignacion_brad['credito'])
asignacion_cred.head(3)

,unegocio_id,credito,division,saldo_recuperar
1,1,CR-101922,COBRANZA EXTRAJUDICIAL,35606.11
2,1,CR-101949,LIQUIDAR,28340.31
3,1,CR-101952,LIQUIDAR,21259.41


In [21]:
#Obtenemos datos de las promesas de Liverpool
pivot_cred = pd.pivot_table(asignacion_cred,index=['division'],values=['credito','saldo_recuperar'],aggfunc=['count',np.sum],margins=True,dropna=False)
pivot_cred = pd.DataFrame(pivot_cred.to_records())
pivot_cred = pivot_cred.iloc[:,[0,1,3]]
pivot_cred.columns = ['Division','NumeroPromesas','SaldoRecuperar']
pivot_cred.fillna(0,inplace=True)
#pivot_cred[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_liv[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_cred

,Division,NumeroPromesas,SaldoRecuperar
0,,1301,4.091192e+07
1,COBRANZA DUAL,8,3.377336e+05
2,COBRANZA EXTRAJUDICIAL,6039,2.326000e+08
3,COBRANZA PAGO CERO,84,5.793293e+06
4,EXTRAJUDICIAL,510,2.174625e+07
5,LIQUIDAR,846,3.433660e+07
6,PREVENTIVA,1,3.465500e+02
7,All,8789,3.357262e+08


# Bradesco

In [23]:
#Hacemos la consulta de la asignacion
sql_asig_brad = cursor.callproc('bradescard_rpt_asignacion_activa',)
for result in cursor.stored_results():
    asignacion_brad = pd.DataFrame(result.fetchall())
asignacion_brad = asignacion_brad.iloc[:,[0,1,3,24,6,26,8]]
asignacion_brad.columns = ['firma_id','unegocio_id','credito','estado','rfc','division','saldo_recuperar']
asignacion_brad['credito'] = pd.to_numeric(asignacion_brad['credito'])
asignacion_brad.head(3)

,firma_id,unegocio_id,credito,estado,rfc,division,saldo_recuperar
0,1,3,2869000138080177,YUCATAN,JURM8405047J801,C0 5PV,974.00
1,1,3,2869000240445953,VERACRUZ,TOHI650826EZ601,C0 4PV,962.00
2,1,3,2869000320452929,PUEBLA,HERL821006J1401,C0 4PV,2539.00


In [25]:
#Obtenemos datos de las promesas de Liverpool
asignacion_brad['saldo_recuperar'] = pd.to_numeric(asignacion_brad['saldo_recuperar'])
pivot_brad = pd.pivot_table(asignacion_brad,index=['division'],values=['credito','saldo_recuperar'],aggfunc=['count',np.sum],margins=True,dropna=False)
pivot_brad = pd.DataFrame(pivot_brad.to_records())
pivot_brad = pivot_brad.iloc[:,[0,1,4]]
pivot_brad.columns = ['Division','NumeroPromesas','SaldoRecuperar']
pivot_brad.fillna(0,inplace=True)
#pivot_cred[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_liv[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_brad

,Division,NumeroPromesas,SaldoRecuperar
0,A,129184,8.673014e+07
1,B,30860,2.150932e+07
2,C0 4PV,4382,1.767318e+07
3,C0 5PV,3861,1.409293e+07
4,C0 6PV,4166,1.452120e+07
5,C1,14567,5.026947e+07
6,C2,11028,3.791186e+07
7,C3,7937,6.010358e+06
8,C4,6890,5.147056e+06
9,All,212875,2.538655e+08


In [26]:
writer = pd.ExcelWriter('/home/estadistico/Documents/Dic 2020.xlsx', engine='xlsxwriter')
pivot_liv.to_excel(writer,'Liverpool',index=False,header=True)
pivot_cred.to_excel(writer,'Credifiel',index=False,header=True)
pivot_brad.to_excel(writer,'Bradesco',index=False,header=True)

writer.save()
writer.close()